In [1]:
articles = open('article.txt', 'r')

In [2]:
d = articles.readlines()

In [3]:
import numpy as np
dictionary = []
for _ in d:
    a = _.split()
    if a == '|':
        continue
    for __ in a:
        dictionary.append(__)

In [4]:
dictionary = np.unique(dictionary)
int_to_dict = dictionary

In [5]:
dict_to_int = {}
for _ in range(dictionary.size):
    dict_to_int[str(dictionary[_])] = [_]

In [6]:
dictionary.size

37936

In [7]:
WINDOW_SIZE = 2
cluster_array = []
for line in d:
    a = line.split()
    for a_index, word in enumerate(a):
        _ = a[ max( a_index-WINDOW_SIZE, 0 ) : min( a_index + WINDOW_SIZE, len(a)+1) ]
        for __ in _:
            if __ != word:
                cluster_array.append([word, __])

In [8]:
len(cluster_array)

778418

In [57]:
def make_one_hot(index, vocab_size):
    ret_array = np.zeros((vocab_size,1))
    ret_array[index] = 1
    return ret_array

In [58]:
make_one_hot(dict_to_int['अ'], dictionary.size)

array([[1.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [59]:
# make x and y_label array
def create_xy_vector(array):
    x = make_one_hot(dict_to_int[array[0]], dictionary.size)
    y = make_one_hot(dict_to_int[array[1]], dictionary.size)
    return x,y

In [60]:
print(create_xy_vector(cluster_array[100]))

(array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]]), array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]]))


In [61]:
import tensorflow as tf

In [62]:
x_input = []
y_label =  []
for _ in range(100):
    (x, y) = create_xy_vector(cluster_array[_])
    x_input.append(x)
    y_label.append(y)

In [63]:

# print(x_input.size)
# x_input = np.asarray(x_input)
# y_label = np.asarray(y_label)

In [64]:
# print(x_input[0])
dictionary.size

37936

In [65]:
vocab_size = dictionary.size
x_ = tf.placeholder(tf.float32, shape=(vocab_size, None))
y_ = tf.placeholder(tf.float32, shape=(vocab_size, None))

In [66]:
EMBEDDING_DIM = 20
w1 = tf.Variable(tf.random_normal([ vocab_size, EMBEDDING_DIM ]))
b1 = tf.Variable(tf.random_normal( [ EMBEDDING_DIM ]))

hidden_representation = tf.add(tf.matmul(x_, w1), b1)

w2 = tf.Variable(tf.random_normal( [EMBEDDING_DIM, vocab_size ] ))
b2 = tf.Variable(tf.random_normal( [vocab_size] ))

prediction = tf.add(tf.matmul(hidden_representation, w2), b2)

In [67]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [68]:
# loss function 
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(prediction), reduction_indices=[1] ))
gradient_descent = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

In [ ]:
for i in range(1000):
    (x, y) = create_xy_vector(cluster_array[i])
    sess.run(gradient_descent, feed_dict={ x_ : x, y_: y})
    print('loss is : ', sess.run(cross_entropy, feed_dict={x_: x, y_: y }))